In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob

import pandas as pd

In [3]:
big_data = pd.DataFrame()
files = glob.iglob('backtests/binance/' + '**/backtest_result.txt', recursive=True)
i = 0
for filename in files:
    #print(f'parsing : {filename}')
    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': filename.replace('backtests/binance','')}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())

        #d['adgt_%'] = d['adgl_%']+d['adgs_%']
        big_data = big_data.append(d, ignore_index=True)

big_data

C:\Users\skhan\AppData\Local\Temp/ipykernel_584628/2803701148.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_584628/2803701148.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_584628/2803701148.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_584628/2803701148.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_

,file,v,s,ebrl,long,short,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrs,tgl_%,tgs_%
0,\1INCHUSDT\plots\2022-02-03T232328\backtest_re...,5.3,1INCHUSDT,0.909573,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,\1INCHUSDT\plots\2022-02-03T235304\backtest_re...,5.3,1INCHUSDT,0.114573,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,\1INCHUSDT\plots\2022-02-03T235442\backtest_re...,5.3,1INCHUSDT,0.838780,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,\1INCHUSDT\plots\2022-02-04T143331\backtest_re...,5.3,1INCHUSDT,0.952475,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,\1INCHUSDT\plots\2022-02-04T144601\backtest_re...,5.3,1INCHUSDT,0.242079,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,\ZECUSDT\plots\2022-02-22T134640\backtest_resu...,5.4,ZECUSDT,0.966600,True,True,0.0600,0.0179,100.0,100.0,0.9792,12.530,3.573
649,\ZECUSDT\plots\2022-02-25T222007\backtest_resu...,5.5,ZECUSDT,0.966000,True,True,0.0599,0.0173,100.0,100.0,0.9792,12.520,3.435
650,\ZECUSDT\plots\2022-02-25T232208\backtest_resu...,5.5,ZECUSDT,0.966000,True,True,0.0599,0.0173,100.0,100.0,0.9792,12.520,3.435
651,\ZENUSDT\plots\2022-03-05T072906\backtest_resu...,5.5,ZENUSDT,0.920900,True,True,0.0372,0.0882,100.0,100.0,0.8434,8.481,19.940


Now filter the dataframe

In [4]:
allowed_symbols = ["ETHUSDT", "XRPUSDT", "1000SHIBUSDT", "DOGEUSDT", "ADAUSDT",
                   "BNBUSDT", "MATICUSDT", "DOTUSDT", "SANDUSDT", "FTMUSDT",
                   "LTCUSDT", "LINKUSDT", "MANAUSDT", "ETCUSDT", "EOSUSDT",
                   "ATOMUSDT", "FILUSDT", "ICPUSDT", "ALICEUSDT", "LRCUSDT",
                   "BCHUSDT", "SUSHIUSDT", "XTZUSDT", "THETAUSDT", "CRVUSDT",
                   "ALGOUSDT", "CHZUSDT", "TRXUSDT", "VETUSDT", "CHRUSDT",
                   "ONEUSDT", "OMGUSDT", "XLMUSDT", "RUNEUSDT", "ENJUSDT",
                   "CELRUSDT", "SXPUSDT", "1INCHUSDT", "SRMUSDT", "COTIUSDT",
                   "ZECUSDT", "QTUMUSDT", "GRTUSDT", "DENTUSDT", "SFPUSDT",
                   "STORJUSDT", "IOTAUSDT", "HOTUSDT", "NEOUSDT", "COMPUSDT",
                   "LINAUSDT", "KAVAUSDT", "BATUSDT", "WAVESUSDT", "IOSTUSDT",
                   "ALPHAUSDT", "RLCUSDT", "HBARUSDT", "DODOUSDT", "XMRUSDT",
                   "SNXUSDT", "ANKRUSDT", "DASHUSDT", "REEFUSDT", "ZENUSDT",
                   "CVCUSDT", "RENUSDT", "ICXUSDT", "RSRUSDT", "SKLUSDT",
                   "NKNUSDT", "BELUSDT", "ONTUSDT", "OGNUSDT", "BLZUSDT",
                   "MTLUSDT", "CTKUSDT", "RVNUSDT", "LITUSDT", "MKRUSDT",
                   "OCEANUSDT", "ZILUSDT", "UNFIUSDT", "BANDUSDT", "YFIIUSDT",
                   "TOMOUSDT", "TRBUSDT", "KNCUSDT", "STMXUSDT", "ZRXUSDT",
                   "SCUSDT", "AKROUSDT", "XEMUSDT", "FLMUSDT", "BALUSDT",
                   "BTSUSDT", "DGBUSDT", "DEFIUSDT"]

allowed_symbols = ["MATICUSDT","MKRUSDT","OCEANUSDT","ZILUSDT","UNFIUSDT"]
big_data['adgt_%'] = big_data['adgl_%'] + big_data['adgs_%']
df = big_data[(big_data.s.isin(allowed_symbols))
              & (big_data['long'] == True)
              & (big_data['short'] == True)
              & (big_data['adgt_%'] > 0.09)
              & (big_data['bnkrl_%'] > 95.0)
              & (big_data['bnkrs_%'] > 95.0)
              & (big_data['ebrs'] > .85)
              & (big_data['ebrl'] > .85)
              & (big_data['v'] == '5.5')

    # & (big_data['tg_%'] > 10)
    # & (big_data['ebr'] > .85)
              ]
#df['total_adg'] = big_data['adgl_%'] + big_data['adgs_%']
df

,file,v,s,ebrl,long,short,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrs,tgl_%,tgs_%,adgt_%
415,\MATICUSDT\plots\2022-02-25T215933\backtest_re...,5.5,MATICUSDT,0.9370,True,True,0.0961,0.0282,100.0,100.0,0.9007,21.430,5.325,0.1243
416,\MATICUSDT\plots\2022-02-25T221119\backtest_re...,5.5,MATICUSDT,0.9447,True,True,0.0837,0.0950,100.0,100.0,0.9575,18.300,20.510,0.1787
417,\MATICUSDT\plots\2022-02-25T230108\backtest_re...,5.5,MATICUSDT,0.9370,True,True,0.0961,0.0282,100.0,100.0,0.9007,21.430,5.325,0.1243
418,\MATICUSDT\plots\2022-02-25T231304\backtest_re...,5.5,MATICUSDT,0.9447,True,True,0.0837,0.0950,100.0,100.0,0.9575,18.300,20.510,0.1787
419,\MATICUSDT\plots\2022-03-03T122701\backtest_re...,5.5,MATICUSDT,0.9333,True,True,0.0863,0.0688,100.0,100.0,0.9140,21.870,16.130,0.1551
420,\MATICUSDT\plots\2022-03-06T075018\backtest_re...,5.5,MATICUSDT,0.9358,True,True,0.1080,0.0408,100.0,100.0,0.8685,26.030,9.620,0.1488
421,\MKRUSDT\plots\2022-03-06T075645\backtest_resu...,5.5,MKRUSDT,0.9503,True,True,0.0423,0.0807,100.0,100.0,0.9048,9.628,19.450,0.1230
432,\OCEANUSDT\plots\2022-03-06T080520\backtest_re...,5.5,OCEANUSDT,0.9442,True,True,0.0895,0.0597,100.0,100.0,0.8995,21.220,14.150,0.1492
580,\UNFIUSDT\plots\2022-03-06T081556\backtest_res...,5.5,UNFIUSDT,0.9276,True,True,0.0982,0.0770,100.0,100.0,0.9453,23.310,22.370,0.1752
652,\ZILUSDT\plots\2022-03-06T080955\backtest_resu...,5.5,ZILUSDT,0.9544,True,True,0.0533,0.0657,100.0,100.0,0.9504,12.050,15.840,0.1190


In [5]:
df.drop_duplicates(subset=['s', 'tgl_%','tgs_%','adgl_%','adgs_%','adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',
                   inplace=True)
dx = df[['file','s','adgt_%','adgl_%','adgs_%','bnkrl_%', 'bnkrs_%','ebrl', 'ebrs']]
dx

C:\Users\skhan\AppData\Local\Temp/ipykernel_584628/2043545472.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=['s', 'tgl_%','tgs_%','adgl_%','adgs_%','adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',


,file,s,adgt_%,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs
415,\MATICUSDT\plots\2022-02-25T215933\backtest_re...,MATICUSDT,0.1243,0.0961,0.0282,100.0,100.0,0.9370,0.9007
416,\MATICUSDT\plots\2022-02-25T221119\backtest_re...,MATICUSDT,0.1787,0.0837,0.0950,100.0,100.0,0.9447,0.9575
419,\MATICUSDT\plots\2022-03-03T122701\backtest_re...,MATICUSDT,0.1551,0.0863,0.0688,100.0,100.0,0.9333,0.9140
420,\MATICUSDT\plots\2022-03-06T075018\backtest_re...,MATICUSDT,0.1488,0.1080,0.0408,100.0,100.0,0.9358,0.8685
421,\MKRUSDT\plots\2022-03-06T075645\backtest_resu...,MKRUSDT,0.1230,0.0423,0.0807,100.0,100.0,0.9503,0.9048
432,\OCEANUSDT\plots\2022-03-06T080520\backtest_re...,OCEANUSDT,0.1492,0.0895,0.0597,100.0,100.0,0.9442,0.8995
580,\UNFIUSDT\plots\2022-03-06T081556\backtest_res...,UNFIUSDT,0.1752,0.0982,0.0770,100.0,100.0,0.9276,0.9453
652,\ZILUSDT\plots\2022-03-06T080955\backtest_resu...,ZILUSDT,0.1190,0.0533,0.0657,100.0,100.0,0.9544,0.9504
